In [1]:
# python
from pathlib import Path
import json

# database
import sqlite3

# data analysis
import pandas as pd
import geopandas as gpd

# geo
from shapely.geometry import Point

In [2]:
current_dir = Path.cwd()
parent_dir = current_dir.parent

In [3]:
conn = sqlite3.connect(parent_dir / 'data' / 'caf_accessibility.db')

def get_dataframe_from_sqlite_db(table_name: str, conn: sqlite3.Connection=conn, geo_type: str='polygon'):
    # columns = utils.concat_list()   
    columns = '*'
    df = pd.read_sql(f'SELECT {columns} FROM {table_name}', conn)
    if geo_type == 'polygon':
        geo = get_geo_from_sqlite_db(table_name, conn)
    elif geo_type == 'point':
        geo = get_geo_from_sqlite_db(table_name, conn)
        features = geo['features']
        lat = list()
        lon = list()
        for f in features:
            lat.append(f['geometry']['coordinates'][1])
            lon.append(f['geometry']['coordinates'][0])
        geo = pd.DataFrame(data={'lat': lat, 'lon': lon})
    return df, geo

def get_geo_from_sqlite_db(table_name: str, conn: sqlite3.Connection=conn):
    c = conn.cursor()
    c.execute(f'SELECT geometry FROM Geos WHERE table_ = "{table_name}"')
    result = c.fetchone()[0]
    return json.loads(result)

# Bogotá

## Movilidad

In [4]:
bta_encuesta_hogares = pd.read_csv(
    parent_dir / "data" / "encuesta_movilidad" / "bogota" / "HogaresEODH2019.csv", sep=";"
)
bta_encuesta_personas = pd.read_csv(
    parent_dir / "data" / "encuesta_movilidad" / "bogota" / "PersonasEODH2019.csv", sep=";"
)
bta_encuesta_viajes = pd.read_csv(
    parent_dir / "data" / "encuesta_movilidad" / "bogota" / "ViajesEODH2019.csv", sep=";"
)

/var/folders/0r/zp2mz7zj28xdnvbwvnqwf8vw0000gn/T/ipykernel_12800/561591205.py:1: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  bta_encuesta_hogares = pd.read_csv(


In [5]:
temp = bta_encuesta_viajes[["id_hogar", "id_persona", "modo_principal", "id_viaje"]].groupby(["id_hogar", "id_persona", "modo_principal"]).count().reset_index()
temp = temp.rename(columns={"id_viaje": "count"})
temp["composite_id"] = temp["id_hogar"].astype(str) + "_" + temp["id_persona"].astype(str)
temp.sort_values(by="count", ascending=False, inplace=True)
temp.drop_duplicates(subset="composite_id", keep="first", inplace=True)

In [6]:
temp.head()

,id_hogar,id_persona,modo_principal,count,composite_id
18909,18803,2,A pie,11,18803_2
63929,P04963,2,A pie,10,P04963_2
54697,P00860,2,Auto,9,P00860_2
9577,14592,1,Auto,9,14592_1
68828,P08648,2,A pie,9,P08648_2


## Encuesta hogares

In [7]:
temp = temp.merge(
    bta_encuesta_hogares[['Id_Hogar', 'Latitud', 'Longitud']],
    left_on='id_hogar',
    right_on='Id_Hogar',
    how='left'
)[['id_hogar', 'id_persona', 'modo_principal', 'composite_id', 'Latitud', 'Longitud']]

In [8]:
def f(x):
    """Transforms lat and long coordinates to Point object."""
    try:
        longitud = float(str(x["Longitud"]).replace(",", "."))
        latitud = float(str(x["Latitud"]).replace(",", "."))
    except:
        print(longitud)
    point = Point(longitud, latitud)
    return point
temp["geometry"] = temp.apply(f, axis=1)
temp.drop(["Latitud", "Longitud"], axis=1, inplace=True)

In [9]:
temp.head()

,id_hogar,id_persona,modo_principal,composite_id,geometry
0,18803,2,A pie,18803_2,POINT (-74.2136983 4.5735933)
1,P04963,2,A pie,P04963_2,POINT (-74.0863301815 4.75262196411)
2,P00860,2,Auto,P00860_2,POINT (-74.0501974176 4.85657176123)
3,14592,1,Auto,14592_1,POINT (-74.267206 4.7261872)
4,P08648,2,A pie,P08648_2,POINT (-74.0380889119 4.76412182479)


In [10]:
temp.modo_principal.value_counts()

A pie                            19037
TransMilenio                      7178
Auto                              6839
SITP Zonal                        4125
Bicicleta                         3164
SITP Provisional                  2457
Moto                              2387
Transporte publico individual     1921
Transporte informal               1684
Transporte Escolar                1664
Intermunicipal                    1492
Otro                               784
Alimentador                        660
Bicitaxi                            55
Patineta                            33
Cable                                1
Name: modo_principal, dtype: int64

## Encuesta personas

In [11]:
temp2 = bta_encuesta_personas[["id_hogar", "id_persona", "Sexo", "p8_id_dificultad_fisica_1", "p8_id_dificultad_fisica_2"]].copy()
temp2["composite_id"] = temp2["id_hogar"].astype(str) + "_" + temp2["id_persona"].astype(str)

In [12]:
temp2.head()

,id_hogar,id_persona,Sexo,p8_id_dificultad_fisica_1,p8_id_dificultad_fisica_2,composite_id
0,10005,2,Mujer,0.0,0.0,10005_2
1,10009,2,Mujer,0.0,0.0,10009_2
2,10015,2,Mujer,0.0,0.0,10015_2
3,10017,2,Mujer,0.0,0.0,10017_2
4,10006,2,Mujer,1.0,0.0,10006_2


In [13]:
temp = temp.merge(temp2[['Sexo', 'p8_id_dificultad_fisica_1', 'p8_id_dificultad_fisica_2', 'composite_id']], how="left", on="composite_id")

In [14]:
temp.head()

,id_hogar,id_persona,modo_principal,composite_id,geometry,Sexo,p8_id_dificultad_fisica_1,p8_id_dificultad_fisica_2
0,18803,2,A pie,18803_2,POINT (-74.2136983 4.5735933),Hombre,0.0,0.0
1,P04963,2,A pie,P04963_2,POINT (-74.0863301815 4.75262196411),Mujer,0.0,0.0
2,P00860,2,Auto,P00860_2,POINT (-74.0501974176 4.85657176123),Mujer,0.0,0.0
3,14592,1,Auto,14592_1,POINT (-74.267206 4.7261872),Hombre,0.0,0.0
4,P08648,2,A pie,P08648_2,POINT (-74.0380889119 4.76412182479),Mujer,0.0,0.0


## Obtener valores para la aplicación

Valores a obtener: peatón, bicicleta, mujer, movilidad de cuidado

In [15]:
# Peatones
def f(x):
    if x=="A pie":
        return 1
    return 0
temp["peaton"] = temp["modo_principal"].apply(f)

In [16]:
# Bicicleta
def f(x):
    if x=="Bicicleta":
        return 1
    return 0
temp["bicicleta"] = temp["modo_principal"].apply(f)

In [17]:
# Mujer
def f(x):
    if x=="Mujer":
        return 1
    return 0
temp["mujer"] = temp["Sexo"].apply(f)

In [18]:
# Movilidad de cuidado
def f(x):
    if x>0:
        return 1
    return 0
temp["movilidad_cuidado"] = temp["p8_id_dificultad_fisica_1"] + temp["p8_id_dificultad_fisica_2"]
temp["movilidad_cuidado"] = temp["movilidad_cuidado"].apply(f)

In [19]:
temp = temp[['peaton', 'bicicleta', 'mujer', 'movilidad_cuidado', 'geometry']]
temp.head()

,peaton,bicicleta,mujer,movilidad_cuidado,geometry
0,1,0,0,0,POINT (-74.2136983 4.5735933)
1,1,0,1,0,POINT (-74.0863301815 4.75262196411)
2,0,0,1,0,POINT (-74.0501974176 4.85657176123)
3,0,0,0,0,POINT (-74.267206 4.7261872)
4,1,0,1,0,POINT (-74.0380889119 4.76412182479)


In [20]:
temp = gpd.GeoDataFrame(temp, geometry="geometry")

## Accesibilidad

In [21]:
accessibility_df, accessibility_geo = get_dataframe_from_sqlite_db(
    table_name='Accessibility', conn=conn)
accessibility_geo = gpd.GeoDataFrame.from_features(accessibility_geo['features'])
accessibility_df['geometry'] = accessibility_geo.geometry
accessibility_df = gpd.GeoDataFrame(accessibility_df, geometry='geometry')

In [22]:
temp = gpd.sjoin(temp, accessibility_df[['geometry', 'hex']], how='left', op='within').reset_index(drop=True)

/Users/cristiantorres/Desktop/work/caf/caf-accessibility-dashboard/env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [23]:
temp.isnull().sum()

peaton                  0
bicicleta               0
mujer                   0
movilidad_cuidado       0
geometry                0
index_right          5161
hex                  5161
dtype: int64

In [24]:
temp.dropna(subset=['hex'], inplace=True)
temp = temp[['peaton', 'bicicleta', 'mujer', 'movilidad_cuidado', 'hex']]

In [25]:
temp.head()

,peaton,bicicleta,mujer,movilidad_cuidado,hex
0,1,0,0,0,8966e09accbffff
1,1,0,1,0,8966e428197ffff
2,0,0,1,0,8966e4389dbffff
4,1,0,1,0,8966e42ad4fffff
7,1,0,1,1,8966e092e67ffff


In [26]:
temp = temp.groupby("hex").sum().reset_index()
temp

,hex,peaton,bicicleta,mujer,movilidad_cuidado
0,8966e082003ffff,10,0,12,0
1,8966e082007ffff,20,0,23,1
2,8966e082013ffff,9,0,9,1
3,8966e08201bffff,7,0,7,0
4,8966e082033ffff,1,0,1,0
...,...,...,...,...,...
2430,8966e466ea7ffff,12,6,13,1
2431,8966e466eafffff,4,3,12,1
2432,8966e466eb3ffff,4,1,10,0
2433,8966e466ecbffff,5,11,10,0


## Unir accesibilidad con datos de encuesta de movilidad

In [28]:
accessibility_df = accessibility_df.merge(
    temp,
    on='hex',
    how='left'
)

In [29]:
accessibility_df.head()

,hex,Poblacion,PCA,area_m2,Densidad_ha,NSE_5,NSE_3,hex_o,AllTrips_AvgTravelTimeAll_tpubl,AllTrips_AvgTravelTimeAll_tpubl_w,...,Verde_m2habitante800m,Verde_m2habitante1500m,Verde_m2habitante2000m,city,IndiAcce_1,geometry,peaton,bicicleta,mujer,movilidad_cuidado
0,8966e4289b7ffff,1477.0,-3.888163,109517.0,134.9,1 - Alto,1 - Alto,8866e4289bfffff,54.35,44.09,...,3.530855,3.767326,3.897198,Bogotá,1. Alta,"POLYGON ((-74.04566 4.69746, -74.04673 4.69601...",5.0,1.0,10.0,0.0
1,8966e4289b3ffff,1505.0,-3.770667,109517.0,137.4,1 - Alto,1 - Alto,8866e4289bfffff,54.35,44.09,...,3.530855,3.767326,3.897198,Bogotá,1. Alta,"POLYGON ((-74.04832 4.69940, -74.04940 4.69794...",NaN,NaN,NaN,NaN
2,8966e4289a7ffff,1364.0,-3.760292,109522.0,124.5,1 - Alto,1 - Alto,8866e4289bfffff,54.35,44.09,...,3.530855,3.767326,3.897198,Bogotá,1. Alta,"POLYGON ((-74.04594 4.69431, -74.04701 4.69285...",2.0,1.0,5.0,0.0
3,8966e4289a3ffff,1380.0,-3.755640,109522.0,126.0,1 - Alto,1 - Alto,8866e4289bfffff,54.35,44.09,...,3.530855,3.767326,3.897198,Bogotá,1. Alta,"POLYGON ((-74.04860 4.69624, -74.04967 4.69479...",3.0,0.0,3.0,0.0
4,8966e4289bbffff,1232.0,-3.705080,109522.0,112.5,1 - Alto,1 - Alto,8866e4289bfffff,54.35,44.09,...,3.530855,3.767326,3.897198,Bogotá,1. Alta,"POLYGON ((-74.05127 4.69818, -74.05234 4.69672...",0.0,0.0,0.0,0.0


In [31]:
accessibility_df.to_parquet(parent_dir / "temp_acc_.parquet")

In [36]:
accessibility_df.movilidad_cuidado.sum()

1744.0

In [41]:
for c in accessibility_df.columns:
    if "Time" in c:
        print(c)


AllTrips_AvgTravelTimeAll_tpubl
AllTrips_AvgTravelTimeAll_tpubl_w
AllTrips_AvgTravelTimeAll_car
AllTrips_AvgTravelTimeAll_car_w
AllTrips_TravelTimeDowntown_tpubl
AllTrips_TravelTimeDowntown_car
AllTrips_TravelTimeDowntown_Difference
EducInicPriv_AvgTime_bus
EducInicPriv_AvgTime_walk
EducInicPubl_AvgTime_bus
EducInicPubl_AvgTime_walk
EducPrimPriv_AvgTime_bus
EducPrimPriv_AvgTime_walk
EducPrimPubl_AvgTime_bus
EducPrimPubl_AvgTime_walk
EducSecuPriv_AvgTime_bus
EducSecuPriv_AvgTime_walk
EducSecuPubl_AvgTime_bus
EducSecuPubl_AvgTime_walk
SaluAtenPrim_AvgTime_bus
SaluAtenPrim_AvgTime_walk
SaluHosp_AvgTime_bus
SaluHosp_AvgTime_walk


In [46]:
(accessibility_df.EducInicPubl_AvgTime_bus < accessibility_df.EducInicPubl_AvgTime_walk).describe()

count      9142
unique        2
top       False
freq       6281
dtype: object

In [49]:
accessibility_df.columns

Index(['hex', 'Poblacion', 'PCA', 'area_m2', 'Densidad_ha', 'NSE_5', 'NSE_3',
       'hex_o', 'AllTrips_AvgTravelTimeAll_tpubl',
       'AllTrips_AvgTravelTimeAll_tpubl_w', 'AllTrips_AvgTransfersAll_tpubl',
       'AllTrips_AvgTransfersAll_tpubl_w', 'AllTrips_AvgTravelTimeAll_car',
       'AllTrips_AvgTravelTimeAll_car_w', 'AllTrips_TravelTimeDowntown_tpubl',
       'AllTrips_AvgTransfersDowntown_tpubl',
       'AllTrips_TravelTimeDowntown_car', 'AllTrips_DistanceDowntown_car',
       'AllTrips_TravelTimeDowntown_Difference', 'rate_tp_car_all',
       'rate_tp_car_Downtown', 'AllTrips_walk_to_bus_origin_meters_w',
       'AllTrips_walk_to_bus_total_meters_w', 'EsVe_Area_verdes_ha_2km',
       'EsVe_Area_verdes_ha_800mts', 'EducInicPriv_QtAvgEst2km',
       'EducInicPriv_QtAvgEst800mts', 'EducInicPriv_QtAvgEst1500mts',
       'EducInicPriv_AvgTime_bus', 'EducInicPriv_AvgTime_walk',
       'EducInicPubl_QtAvgEst2km', 'EducInicPubl_QtAvgEst800mts',
       'EducInicPubl_QtAvgEst1500mts', '

In [ ]:
bta_encuesta_personas = bta_encuesta_personas.merge(
    bta_encuesta_hogares[['Id_Hogar', 'Latitud', 'Longitud']],
    left_on='id_hogar',
    right_on='Id_Hogar',
    how='left'
)

In [ ]:
bta_encuesta_personas.shape

In [ ]:
def f(x):
    """Transforms lat and long coordinates to Point object."""
    try:
        longitud = float(str(x["Longitud"]).replace(",", "."))
        latitud = float(str(x["Latitud"]).replace(",", "."))
    except:
        print(longitud)
    point = Point(longitud, latitud)
    return point
bta_encuesta_personas["geometry"] = bta_encuesta_personas.apply(f, axis=1)

In [ ]:
bta_encuesta_personas = gpd.GeoDataFrame(bta_encuesta_personas, geometry="geometry")

In [ ]:
bta_encuesta_personas.head()

## Accesibilidad

In [ ]:
accessibility_df, accessibility_geo = get_dataframe_from_sqlite_db(
    table_name='Accessibility', conn=conn)
accessibility_geo = gpd.GeoDataFrame.from_features(accessibility_geo['features'])
accessibility_df['geometry'] = accessibility_geo.geometry
accessibility_df = gpd.GeoDataFrame(accessibility_df, geometry='geometry')

## Merge movilidad and accessibility

In [ ]:
temp = gpd.sjoin(bta_encuesta_personas, accessibility_df[['geometry', 'hex']], how='left', op='within').reset_index(drop=True)


In [ ]:
temp.head()

In [ ]:
temp.shape

In [ ]:
temp.hex.isnull().sum()

In [ ]:
temp.dropna(subset=['hex'], inplace=True)

## Variables que caracterizan la dificultad de medios de transporte

* p9_id_dificultad_medios_transporte_1	(Dificultades para utilizar medios de transporte)
* p9_id_dificultad_medios_transporte_2	(Dificultad para usar automóvil)
* p9_id_dificultad_medios_transporte_3	(Dificultad para usar motocicleta)
* p9_id_dificultad_medios_transporte_4	(Dificultad para usar bus/buseta/microbús)
* p9_id_dificultad_medios_transporte_5	(Dificultad para usar TransMilenio)
* p9_id_dificultad_medios_transporte_6	(Dificultad para usar TransMiCable)
* p9_id_dificultad_medios_transporte_7	(Dificultad para usar Sitp)
* p9_id_dificultad_medios_transporte_8	(Dificultad para usar taxi)
* p9_id_dificultad_medios_transporte_9	(Dificultad para usar campero/jeep)
* p9_id_dificultad_medios_transporte_10	(Dificultad para usar bus escalera/chiva)
* p9_id_dificultad_medios_transporte_11	(Dificultad para usar bicicleta)
* p9_id_dificultad_medios_transporte_12	(Dificultad para usar bicitaxi)
* p9_id_dificultad_medios_transporte_13	(Dificultad para usar todos los anteriores)

### p9_id_dificultad_medios_transporte_2

In [ ]:
p_auto = temp[['p9_id_dificultad_medios_transporte_2', 'hex']].copy()

In [ ]:
p_auto.isnull().sum()

In [ ]:
p_auto.p9_id_dificultad_medios_transporte_2 = p_auto.p9_id_dificultad_medios_transporte_2.fillna(0)

In [ ]:
p_auto.isnull().sum()

In [ ]:
p_auto = p_auto.groupby('hex').sum()

In [ ]:
p_auto.head()

### p9_id_dificultad_medios_transporte_11

In [ ]:
p_bici = temp[['p9_id_dificultad_medios_transporte_11', 'hex']].copy()


In [ ]:
p_bici.isnull().sum()

In [ ]:
p_bici.p9_id_dificultad_medios_transporte_11 = p_bici.p9_id_dificultad_medios_transporte_11.fillna(0)

In [ ]:
p_bici.isnull().sum()

In [ ]:
p_bici = p_bici.groupby('hex').sum()

In [ ]:
p_bici.head()

### p9_id_dificultad_medios_transporte_4

In [ ]:
p_bus = temp[['p9_id_dificultad_medios_transporte_4', 'hex']].copy()

In [ ]:
p_bus.isnull().sum()

In [ ]:
p_bus.p9_id_dificultad_medios_transporte_4 = p_bus.p9_id_dificultad_medios_transporte_4.fillna(0)


In [ ]:
p_bus.isnull().sum()

In [ ]:
p_bus = p_bus.groupby('hex').sum()

In [ ]:
p_bus.head()

### p9_id_dificultad_medios_transporte_5

In [ ]:
p_transmilenio = temp[['p9_id_dificultad_medios_transporte_5', 'hex']].copy()

In [ ]:
p_transmilenio.isnull().sum()

In [ ]:
p_transmilenio.p9_id_dificultad_medios_transporte_5 = p_transmilenio.p9_id_dificultad_medios_transporte_5.fillna(0)

In [ ]:
p_transmilenio.isnull().sum()

In [ ]:
p_transmilenio = p_transmilenio.groupby('hex').sum()

In [ ]:
poblacion_vulnerable_geo = pd.concat([p_bus, p_bici, p_auto, p_transmilenio], axis=1).reset_index()

In [ ]:
poblacion_vulnerable_geo.head(2)

In [ ]:
accessibility_df.head(2)

In [ ]:
accessibility_df = accessibility_df.merge(
    poblacion_vulnerable_geo,
    on='hex',
    how='left'
)

In [ ]:
accessibility_df.head(2)

In [ ]:
accessibility_df.to_parquet(parent_dir / "temp_acc_.parquet")

In [ ]:
temp.plot()

In [ ]:
temp.head()

In [ ]:
temp.p9_id_dificultad_medios_transporte_4.value_counts()

In [ ]:
temp.p9_id_dificultad_medios_transporte_4.isnull().sum()

In [ ]:
temp[temp.p9_id_dificultad_medios_transporte_5==1][['geometry']].to_parquet('p9_id_dificultad_medios_transporte_5.parquet')